In [ ]:
# !pip3 uninstall -y tensorflow
# !pip3 install tensorflow==1.15

In [1]:
cd drive/My Drive/cose474/assignment8/

/content/drive/My Drive/cose474/assignment8


In [2]:
#! /usr/bin/env python

import tensorflow as tf
print('TF version:',tf.__version__)
import numpy as np
import os
import smart_open
import pickle
import data_helpers as dh
from text_cnn import TextCNN

TF version: 1.15.0


In [3]:
tf.flags.DEFINE_string('f', '', 'kernel')
def del_all_flags(FLAGS):
  flags_dict = FLAGS._flags()
  keys_list = [keys for keys in flags_dict]
  for keys in keys_list:
    FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [ ]:
del_all_flags(tf.flags.FLAGS)
tf.flags.DEFINE_string('f', '', 'kernel')

In [ ]:

# Parameters
# ==================================================

# Data Parameters
tf.flags.DEFINE_string("trec_test_file", "./data/TREC/testdata.txt", "Data source for the test")
tf.flags.DEFINE_string("mr_test_file_pos", "./data/MR/rt-polarity_test.pos", "Data source for the ODP test")
tf.flags.DEFINE_string("mr_test_file_neg", "./data/MR/rt-polarity_test.neg", "Data source for the ODP test")
tf.flags.DEFINE_string("task", "TREC", "Choose the classification task") #TREC 데이터에 대한 평가

# Eval Parameters
tf.flags.DEFINE_string("dir", "./runs/1605865901", "Checkpoint directory from training run")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")


FLAGS = tf.flags.FLAGS


if FLAGS.task == "MR":
    x_raw, y_test = dh.load_mr_data(FLAGS.mr_test_file_pos, FLAGS.mr_test_file_neg)
elif FLAGS.task == "TREC":
    x_raw, y_test = dh.load_trec_data(FLAGS.trec_test_file) # 구현한 load_trec_data 사용
y_test = np.argmax(y_test, axis=1) # 0, 0, 0, 1 -> 3

# Map data into vocabulary
with smart_open.smart_open(os.path.join(FLAGS.dir, "vocab"), 'rb') as f:
    word_id_dict = pickle.load(f)
with smart_open.smart_open(os.path.join(FLAGS.dir, "config"), 'rb') as f:
    config = pickle.load(f)

    x_test = dh.text_to_index(x_raw, word_id_dict, max(list(map(int, config["filter_sizes"].split(",")))) - 1)
    x_test = dh.test_tensor(x_test, config["max_length"])

print("\nEvaluating...\n")

# Evaluation
# ==================================================
checkpoint_file = tf.train.latest_checkpoint(os.path.join(FLAGS.dir, "checkpoints"))
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(config)
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(tf.global_variables())
        saver.restore(sess, checkpoint_file)

        # Generate batches for one epoch
        batches = dh.batch_iter(list(x_test), config["batch_size"], 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(cnn.predictions, {cnn.input_x: x_test_batch, cnn.dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    print(len(all_predictions))
    print(len(y_test))
    print(sum(all_predictions == y_test))
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

### Inference 구현 실패...파트입니다...

In [5]:
# Preprocess one positive & negative sample
def sample_mr_data(pos_sample, neg_sample):
    pos_text = dh.clean_str(pos_sample)
    neg_text = dh.clean_str(neg_sample)
    positive_labels = [[0, 1]] # 긍정 review 개수만큼 ground_truth 생성 [0, 1]
    negative_labels = [[1, 0]] # 부정 review 개수만큼 ground_truth 생성 [0, 1]
    y = np.concatenate([positive_labels, negative_labels], 0)
    x_final = pos_text + neg_text
    return [x_final, y]
    x_final = pos_text + neg_text
    return [x_final, y]


In [23]:
del_all_flags(tf.flags.FLAGS)
tf.flags.DEFINE_string('f', '', 'kernel')

In [24]:
infer_sample_neg = "This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
infer_sample_pos = 'Sure, this one isn\'t really a blockbuster, nor does it target such a position. "Dieter" is the first name of a quite popular German musician, who is either loved or hated for his kind of acting and thats exactly what this movie is about. It is based on the autobiography "Dieter Bohlen" wrote a few years ago but isn\'t meant to be accurate on that. The movie is filled with some sexual offensive content (at least for American standard) which is either amusing (not for the other "actors" of course) or dumb - it depends on your individual kind of humor or on you being a "Bohlen"-Fan or not. Technically speaking there isn\'t much to criticize. Speaking of me I find this movie to be an OK-movie.'

tf.flags.DEFINE_string("dir", "./runs/1605761020", "Checkpoint directory from training run")
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
FLAGS = tf.flags.FLAGS

x_sample,y_sample = sample_mr_data(infer_sample_pos, infer_sample_neg)
y_sample = np.argmax(y_sample, axis=1) # 0, 0, 0, 1 -> 3
y_sample

with smart_open.smart_open(os.path.join(FLAGS.dir, "vocab"), 'rb') as f:
    word_id_dict = pickle.load(f)
with smart_open.smart_open(os.path.join(FLAGS.dir, "config"), 'rb') as f:
    config = pickle.load(f)
    x_sample = dh.text_to_index(x_sample, word_id_dict, max(list(map(int, config["filter_sizes"].split(",")))) - 1)
    x_sample = dh.test_tensor(x_sample, config["max_length"])

x_sample
#여기서 잘못불러오는 느낌인데... 잘 모르겠습니다...

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:

with smart_open.smart_open(os.path.join(FLAGS.dir, "vocab"), 'rb') as f:
    word_id_dict = pickle.load(f)
with smart_open.smart_open(os.path.join(FLAGS.dir, "config"), 'rb') as f:
    config = pickle.load(f)
    x_sample = dh.text_to_index(x_sample, word_id_dict, max(list(map(int, config["filter_sizes"].split(",")))) - 1)
    x_sample = dh.test_tensor(x_sample, config["max_length"])

checkpoint_file = tf.train.latest_checkpoint(os.path.join(FLAGS.dir, "checkpoints"))
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)

    with sess.as_default():
        cnn = TextCNN(config)
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(tf.global_variables())
        saver.restore(sess, checkpoint_file)
        pred = sess.run(cnn.predictions, {cnn.input_x: x_sample, cnn.dropout_keep_prob: 1.0})
        print("pos:1, neg:0")
        print(all_predictions)

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/cose474/assignment8/runs/1605761020/checkpoints/model-14400
pos:1, neg:0
[1. 0. 0. ... 0. 1. 1.]
